## Imports

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
import pickle

## Creating global containers for holding stats

In [11]:
#sparse_data = lil_matrix((12000, 61188), dtype=np.int16)
#set_list = [set() for x in range(0,20)]
# document_label -> indices in the data
class_row_dict=dict(zip(list(range(1,21)),set_list))
# this tells us which row to look at to get word counts for each class
# sparse_row_leader = [-1] * 21
dense_matrix = np.zeros(shape=(21,61188), dtype=np.int16)

## This has been run before we can simply load the sparse matrix: see_below
### This loads the csv into  dense matrix and then converts it to sparse matrix and saves

In [12]:
with open('training.csv', 'r') as train_strem:
    i=0
    for line in train_strem:
        line_int = np.array(list(map(int, line.split(','))), dtype=np.int16)
        doc_label = line_int[-1]
        dense_matrix[doc_label] += line_int[1:-1]
        i += 1
        print(i)
    sparse_training = sparse.csr_matrix(dense_matrix)
    sparse.save_npz('sparse_training.data',sparse_training)
    print(sparse_training)

401
8402
8403
8404
8405
8406
8407
8408
8409
8410
8411
8412
8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8

NotImplementedError: Save is not implemented for sparse matrix of format lil.

## loading the sparse matrix

In [15]:
#how load
#print(sparse_row_leader)
sparse_data = sparse.load_npz('sparse_training.data.npz')
print(sparse_data)

(1, 0)	9
  (1, 1)	53
  (1, 2)	237
  (1, 3)	11
  (1, 4)	48
  (1, 5)	36
  (1, 6)	7
  (1, 7)	1
  (1, 8)	31
  (1, 9)	127
  (1, 10)	5
  (1, 11)	4336
  (1, 12)	19
  (1, 13)	24
  (1, 14)	46
  (1, 15)	597
  (1, 16)	267
  (1, 17)	12
  (1, 18)	9
  (1, 19)	15
  (1, 20)	2
  (1, 21)	3
  (1, 22)	2985
  (1, 23)	4
  (1, 24)	325
  :	:
  (20, 61146)	1
  (20, 61147)	1
  (20, 61148)	1
  (20, 61149)	2
  (20, 61150)	1
  (20, 61151)	1
  (20, 61152)	1
  (20, 61153)	1
  (20, 61154)	1
  (20, 61169)	2
  (20, 61170)	2
  (20, 61171)	2
  (20, 61172)	3
  (20, 61173)	4
  (20, 61174)	2
  (20, 61175)	3
  (20, 61176)	6
  (20, 61177)	1
  (20, 61178)	2
  (20, 61179)	2
  (20, 61183)	2
  (20, 61184)	2
  (20, 61185)	2
  (20, 61186)	2
  (20, 61187)	2


## Counting Priors and words

In [16]:
word_id_ranges = list(range(1,61189))
column_names =  ['doc_id'] + word_id_ranges + ['label']
prior_counts = [0] * 21
#for data_chunk in pd.read_csv('training.csv', header=None, chunksize=200, names=column_names, usecols=['label']):
i = 0
for data_chunk in pd.read_csv('training.csv', header=None, chunksize=200, names=column_names, usecols=['label']): 
    for _, row in data_chunk.iterrows():
        current_label = row['label']
        class_row_dict[current_label].add(i)
        i += 1
for j in range(1,21):
    prior_counts[j] = len(class_row_dict[j])
print(prior_counts) 

[0, 483, 624, 622, 643, 602, 630, 618, 614, 649, 628, 646, 639, 626, 621, 637, 651, 580, 593, 467, 427]


## Adding the word counts for each class

In [16]:
for doc_cls_id, row_set in class_row_dict.items():
    print(f"class {doc_cls_id} starting")
    row_leader = row_set.pop()
    sparse_row_leader[doc_cls_id] = row_leader
    for other_row in row_set:
        sparse_data[row_leader] += sparse_data[other_row]

class 1 done


KeyboardInterrupt: 

## The cells below are for Experiments only

In [7]:
#building a sparse matrix
with open('training.csv', 'r') as train_strem:
    i = 0
    for line in train_strem:
        line_int = np.array(list(map(int, line.split(','))), dtype=np.int16)
        doc_label = line_int[-1]
        row_leader = sparse_row_leader[doc_label]
        # print(row_leader)
        if row_leader == -1:
            sparse_row_leader[doc_label] = i
            sparse_data[i,:] = line_int[1:-1]
        else:
            sparse_data[row_leader] += line_int[1:-1] 

        # sparse_data[i,:] = list(map(int, line.split(',')))[1:-1]
        i+=1
        print(i)
#saving the sparse matrix to file
sparse_data = sparse_data.tocsr()
sparse.save_npz('sparse.data',sparse_data)
with open('row_leader.obj', 'wb') as picke_stream:
    pickle.dump(row_leader, picke_stream)

401
8402
8403
8404
8405
8406
8407
8408
8409
8410
8411
8412
8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8

In [40]:
# I use index 1-> 61188
count_data=[{'label':x, 'counts':np.zeros(shape=(61189), dtype=np.int32)} for x in range(1,21)]
count_df = pd.DataFrame(count_data, index=list(range(1,21)))
print("this is what the df looks like")
print(count_df.head())
print('This is how we index!')
print(count_df.loc[1,'label'])
print('accessing the numpuy array')
print(type(count_df.loc[1,'counts'][0]))

this is what the df looks like
                                              counts  doc_id
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...       1
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...       2
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...       3
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...       4
5  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...       5
This is how we index!
1
accessing the numpuy array
<class 'numpy.int32'>


In [9]:
# DO NOT RUN THIS CELL BUT KEEP IT

word_id_ranges = list(range(1,61189))
column_names =  ['doc_id'] + word_id_ranges + ['label']
prior_counts = [0] * 21
#for data_chunk in pd.read_csv('training.csv', header=None, chunksize=200, names=column_names, usecols=['label']):
for data_chunk in pd.read_csv('training.csv', header=None, chunksize=200, names=column_names, usecols=['label']): 
    for _, row in data_chunk.iterrows():
        #what document label we are dealing with!?
        current_label = row['label']
        # So we have seen one more of this type!
        prior_counts [current_label] += 1

print(prior_counts)

[0, 483, 624, 622, 643, 602, 630, 618, 614, 649, 628, 646, 639, 626, 621, 637, 651, 580, 593, 467, 427]
